# 文章の類似度によるラベル付与

In [20]:
import pandas as pd

meigen_data = pd.read_pickle("data/meigen.pkl")
meigen_data.head()

,meigen,anime,character
0,「(会話を)どこに投げてんのよ!!」(紅莉栖),シュタインズ・ゲート,[紅莉栖]
1,「4日後、8月15日。お前の死はこの世界によって決められている。避けられやしない！ だが、俺...,シュタインズ・ゲート,[岡部]
2,「70年台だって、悪くないよ。そこでまた君たちみたいな友だちを作って、自由に生きていく」(鈴羽),シュタインズ・ゲート,[鈴羽]
3,「SERNに捕らえられたそのメールを消し、ベータ世界線に戻るということは、つまりその世界に戻...,シュタインズ・ゲート,[岡部]
4,「『漆原るか』。声も仕草も、女にしか見えない。いや、女より女らしい美少女。だが男だ！」(岡部),シュタインズ・ゲート,[岡部]


In [21]:
from transformers import AutoTokenizer, AutoModel
import torch

tokenizer = AutoTokenizer.from_pretrained("pkshatech/GLuCoSE-base-ja")
model = AutoModel.from_pretrained("pkshatech/GLuCoSE-base-ja")

In [22]:
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    embeddings = torch.mean(last_hidden_states, dim=1).tolist()[0]
    return embeddings

meigen_data["meigen_embedding"] = meigen_data["meigen"].apply(get_embedding)

In [23]:
meigen_data.head()

,meigen,anime,character,meigen_embedding
0,「(会話を)どこに投げてんのよ!!」(紅莉栖),シュタインズ・ゲート,[紅莉栖],"[-0.561347484588623, -0.5458228588104248, -0.1..."
1,「4日後、8月15日。お前の死はこの世界によって決められている。避けられやしない！ だが、俺...,シュタインズ・ゲート,[岡部],"[-0.19380775094032288, 0.1803627908229828, 0.3..."
2,「70年台だって、悪くないよ。そこでまた君たちみたいな友だちを作って、自由に生きていく」(鈴羽),シュタインズ・ゲート,[鈴羽],"[0.18892811238765717, -0.04539930820465088, -0..."
3,「SERNに捕らえられたそのメールを消し、ベータ世界線に戻るということは、つまりその世界に戻...,シュタインズ・ゲート,[岡部],"[-0.9506322741508484, 0.37638235092163086, 0.1..."
4,「『漆原るか』。声も仕草も、女にしか見えない。いや、女より女らしい美少女。だが男だ！」(岡部),シュタインズ・ゲート,[岡部],"[-0.09074414521455765, -0.6516573429107666, 0...."


# LDAによるラベル付与

# クラスタリングによるラベル付与